In [1]:
import torch
import torch.nn as nn
import numpy as np

In [151]:
def _income_to_pua(agent_incomes):
    threshold = 304.0*torch.ones_like(agent_incomes)
    pua_amount = torch.min(agent_incomes / 2, threshold)

    return pua_amount

In [ ]:
def get_eligible_pua_payments(payment_amount, shape, request_date, delay=28, frequency=7, num_payments=26):
    agent_pua_payments = payment_amount*torch.zeros_like(shape)

    start_date = request_date + delay
    end_date = request_date + frequency*num_payments

    first_amount = payment_amount*(delay // frequency)
    recurring_amount = payment_amount

    return start_date, end_date, first_amount, recurring_amount

    agent_pua_payments[start_date] = first_amount
    agent_pua_payments[start_date + frequency: end_date: frequency] = recurring_amount

    return agent_pua_payments


In [ ]:
num_agents = 0

start_date_tensor = torch.zeros(())

For each stimulus, we have to do the following:
1. Initialize eligibility
2. Update eligibility
3. If eligible, define payment_amount, payment_frequency, payment_delay

- StimulusPayment
    - Eligibility defined once at start of the simulation
    - Fixed amount for all, one-time payments: Initialized at the start of the simulation
- FPEC
    - Eligiblity defined implicitly for all unemployed people
        - At each step, [eligible_agents = (occupation_status == unemployed)]
    - Fixed amount for all, weekly payments, fixed interval program
- PUA
    - Eligibilty defined on individual request.
        - Initialize simulation with some eligible agents. [eligible = unemployed*request_pua_assistance]
        - During simulation, policy for agents to request assistance.
            - Action: Sample agent eligibility, weekly amount, start date.
    - Variable amount for all, calculated once when eligibility defined.

Tensors to store:
1. Agent_Eligiblity
2. Agent_Stimulus_Amount - (num_agents,)
3. Agent_Stimulus_Dates - (num_steps,)

4. Agent_Stimulus_Payment - (Payment is in-frequent and same for all) - (num_steps,) and sparse_tensor

## Works with torch.sparse_tensor and torch.vmap

In [174]:
# pua_payments tensor
import math

def _income_to_pua(agent_incomes):
    threshold = 304.0*torch.ones_like(agent_incomes)
    pua_amount = torch.min(agent_incomes / 2, threshold)

    return pua_amount

# write for one agent
def get_pua_payments(payment_amount, shape, request_date, delay=3, frequency=7, num_payments=26):
    agent_pua_payments = payment_amount*torch.zeros_like(shape)

    start_date = request_date + delay
    end_date = request_date + frequency*num_payments
    first_amount = payment_amount*math.ceil(delay / frequency)
    recurring_amount = payment_amount

    print(first_amount, recurring_amount)

    agent_pua_payments[start_date] = first_amount
    agent_pua_payments[start_date + frequency: end_date: frequency] = recurring_amount

    return agent_pua_payments

num_agents = 4
num_steps = 23
request_date = 2
agents_income = torch.tensor([200, 800], dtype=torch.float32)
num_steps_shape = torch.ones((num_steps))

pua_payments = torch.zeros((num_agents, num_steps))
sparse_pua_payments = pua_payments.to_sparse_coo()

requesting_agents_indices = torch.tensor([0, 2])
requesting_agents_incomes = torch.tensor([200, 800], dtype=torch.float32)
requesting_payment_amount = _income_to_pua(requesting_agents_incomes)

print("Inputs...")
print("agent payments: ", requesting_payment_amount.shape, requesting_payment_amount)
print("num steps: ", num_steps_shape.shape)
print("-------------")

batched_pua_payments_func = torch.vmap(get_pua_payments, in_dims=(0, None, None))
batched_pua_payments = batched_pua_payments_func(requesting_payment_amount, num_steps_shape, request_date)
sparse_batched_pua_payments = batched_pua_payments.to_sparse_coo()

chunked_batched_pua = batched_pua_payments.chunk(chunks=num_agents, dim=0)
chunked_pua_payments = pua_payments.chunk(chunks=num_agents, dim=0)

print("batched pua payments: ", batched_pua_payments)
print("pua payments", pua_payments)

print("---------------")

pua_payments[requesting_agents_indices] = batched_pua_payments
sparse_pua_payments = pua_payments.to_sparse_coo()

print(sparse_pua_payments)
# sparse_batched_pua_payments = batched_pua_payments.to_sparse_coo()
# print(batched_pua_payments.shape, sparse_batched_pua_payments)

Inputs...
agent payments:  torch.Size([2]) tensor([100., 304.])
num steps:  torch.Size([23])
-------------
BatchedTensor(lvl=1, bdim=0, value=
    tensor([100., 304.])
) BatchedTensor(lvl=1, bdim=0, value=
    tensor([100., 304.])
)
batched pua payments:  tensor([[  0.,   0.,   0.,   0.,   0., 100.,   0.,   0.,   0.,   0.,   0.,   0.,
         100.,   0.,   0.,   0.,   0.,   0.,   0., 100.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0., 304.,   0.,   0.,   0.,   0.,   0.,   0.,
         304.,   0.,   0.,   0.,   0.,   0.,   0., 304.,   0.,   0.,   0.]])
pua payments tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
-------

In [172]:
x1 = torch.zeros((1, 3))
x2 = torch.ones((1, 3))
x3 = torch.randn((1, 3))

x = torch.vstack([x1, x2, x3])

update_indices = [1, 2]
updated_x = x[update_indices] + 100
print(updated_x.shape)

print(x)
print("--------------------")

x_chunk = x.chunk(chunks=3, dim=0)
print(x_chunk[0].shape)

x[update_indices] = updated_x

print(x)

3 // 7

torch.Size([2, 3])
tensor([[ 0.0000,  0.0000,  0.0000],
        [ 1.0000,  1.0000,  1.0000],
        [ 0.2881, -0.4046,  0.0959]])
--------------------
torch.Size([1, 3])
tensor([[  0.0000,   0.0000,   0.0000],
        [101.0000, 101.0000, 101.0000],
        [100.2881,  99.5954, 100.0959]])


0

In [137]:
## debugging torch.vmap design

def _from_x(x):
    return x

def func(x, s):
    # val = _from_x(x)
    # print(val, val.shape)
    ret_val = x*torch.zeros_like(s)
    ret_val[1] = x*5

    ret_val[2: 8: 3] = x - 1

    return ret_val

def func2(agent_payments, num_steps, delay=28, frequency=2):
    agent_pua = agent_payments*torch.zeros_like(num_steps)

    agent_pua[1] = agent_payments + delay
    agent_pua[2: 20: 4] = agent_payments - 2

    return agent_pua

agent_payments = torch.randn((4)) # (num_agents)
num_steps = torch.ones((5)) # (num_steps)

batched_pow = torch.vmap(func2, in_dims=(0, None))

print("-------------")
print("input: ", agent_payments.shape, agent_payments)
print(num_steps.shape, num_steps)
print("-------------")

ans = batched_pow(agent_payments, num_steps)

print("output: ", ans.shape, ans)

-------------
input:  torch.Size([4]) tensor([-0.8818,  1.4248, -0.4376,  0.2385])
torch.Size([5]) tensor([1., 1., 1., 1., 1.])
-------------
output:  torch.Size([4, 5]) tensor([[-0.0000, 27.1182, -2.8818, -0.0000, -0.0000],
        [ 0.0000, 29.4248, -0.5752,  0.0000,  0.0000],
        [-0.0000, 27.5624, -2.4376, -0.0000, -0.0000],
        [ 0.0000, 28.2385, -1.7615,  0.0000,  0.0000]])


## Start PUA for agents during a specific simulation step

In [ ]:
# pua_payments tensor
def _income_to_pua(agent_incomes):
    threshold = 304.0*torch.ones_like(agent_incomes)
    pua_amount = torch.min(agent_incomes / 2, threshold)

    return pua_amount

def get_at_indices(tensor, indices):
    return tensor[indices]

# write for one agent
def get_pua_payments(payment_amount, shape, request_date, delay=3, frequency=7, num_payments=26):
    '''Computes PUA for a single agent'''
    agent_pua_payments = payment_amount*torch.zeros_like(shape)

    start_date = request_date + delay
    end_date = request_date + frequency*num_payments
    first_amount = payment_amount*math.ceil(delay / frequency)
    recurring_amount = payment_amount

    agent_pua_payments[start_date] = first_amount
    agent_pua_payments[start_date + frequency: end_date: frequency] = recurring_amount

    return agent_pua_payments

def start_pua(t, pua_payments, agents_enrollment_status, requesting_pua_agents, agents_income, agents_employment):
    eligible_agents = (agents_employment == 0) # unemployed agents are eligible
    payment_amounts = _income_to_pua(agents_income)

    newly_enrolled_agents = eligible_agents*requesting_pua_agents*(1 - agents_enrollment_status) # all requesting eligible agents receive it if they don't actively do yet
    newly_enrolled_agents_indices = torch.gather(newly_enrolled_agents, dim=1) # gather their indices

    batched_pua_payments_func = torch.vmap(get_pua_payments, in_dims=(0, None, None))
    batched_pua_payments = batched_pua_payments_func(requesting_payment_amount, num_steps_shape, request_date)

    pua_payments[newly_enrolled_agents_indices] = batched_pua_payments
    pua_payments = pua_payments.to_sparse_coo()

    # update enrollment status
    agents_enrollment_status = agents_enrollment_status*(1 - newly_enrolled_agents) + newly_enrolled_agents*(1 - agents_enrollment_status)

    return pua_payments, agents_enrollment_status


pua_payments = torch.zeros((num_agents, num_steps))

requesting_agents_indices = torch.tensor([0, 2])
requesting_agents_incomes = torch.tensor([200, 800], dtype=torch.float32)
requesting_payment_amount = _income_to_pua(requesting_agents_incomes)

print("Inputs...")
print("agent payments: ", requesting_payment_amount.shape, requesting_payment_amount)
print("num steps: ", num_steps_shape.shape)
print("-------------")

batched_pua_payments_func = torch.vmap(get_pua_payments, in_dims=(0, None, None))
batched_pua_payments = batched_pua_payments_func(requesting_payment_amount, num_steps_shape, request_date)
sparse_batched_pua_payments = batched_pua_payments.to_sparse_coo()

chunked_batched_pua = batched_pua_payments.chunk(chunks=num_agents, dim=0)
chunked_pua_payments = pua_payments.chunk(chunks=num_agents, dim=0)

print("batched pua payments: ", batched_pua_payments)
print("pua payments", pua_payments)

print("---------------")

pua_payments[requesting_agents_indices] = batched_pua_payments
sparse_pua_payments = pua_payments.to_sparse_coo()    



def start_pua(t, agents_employment, agents_enrollment_status, agents_request_step, agents_income, num_steps_shape, INFINITY_TIME):
    '''enrollment_status, request_date, delay, start_date, end_date, recurring_amount, first_payment'''
    payment_amount = _income_to_pua(agents_income)
    eligible_agents = (agents_employment == 0)
    ENROLLMENT_VAR = 1

    agents_enrollment_status = agents_enrollment_status*(1-eligible_agents) + ENROLLMENT_VAR*(eligible_agents)
    agents_request_step = (eligible_agents)*t + (1-eligible_agents)*agents_request_step

    batched_pua_payments_func = torch.vmap(get_eligible_pua_payments, in_dims=(0, None, None))
    batched_pua_payments = batched_pua_payments_func(payment_amount, num_steps_shape, agents_request_step)

    sparse_batched_pua_payments = batched_pua_payments.to_sparse_coo()
    print(batched_pua_payments.shape, sparse_batched_pua_payments)

num_agents = 2
INFINITY_TIME = 1000
agents_income = torch.tensor([200, 800], dtype=torch.float32)
agents_employment = torch.tensor([1, 0])
num_steps_shape = torch.ones((100))
agents_enrollment_status = torch.zeros_like(agents_income)
agents_request_step = torch.ones_like(agents_income)*(INFINITY_TIME)



In [143]:
## debug vmap indexing from a mask
x = torch.tensor([1, 2, 3, 4, 5, 6])
torch.select(x, 0, 1)

# explore: torch.chunk
x2 = x.chunk(chunks=2, dim=0)
len(x2)

2

## Key bottleneck: handling masks with differentiability 

In [146]:
# indexing at scale with vmap and sparse_tensors

tensor = torch.tensor([1, 2, 3, 5, 4, 3, 1])
indices = torch.tensor([0, 1, 3])

def get_at_indices(tensor, indices):
    return tensor[indices]

get_at_indices(tensor, indices)


tensor([1, 2, 5])